<h3><b><u>Installing optimization libraries</u></b></h3>

In [1]:
%%capture
!pip install geneticalgorithm pyswarms pulp

<h3><u>Monte Carlo For Estimation of Arrival Rates at the Stations</u></h3>
Apply Monte Carlo for estimating arrival rates at each station locations. 
<br><br>
<table>
    <tr>
        <th>Variables</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>iterations</td>
        <td>Number of polls generated</td>
    </tr>
    <tr>
        <td>stations_lat_longs</td>
        <td>List containing lat-longs of stations in the form [(lat1, long1),(lat2, long2),.....,(latN, longN)]</td>
    </tr>
    <tr>
        <td>center</td>
        <td>lat-longs of the centre of the circle in which random origin and destination lat-longs are generated</td>
    </tr>
    <tr>
        <td>stations</td>
        <td>Dict containing the number of vehicles received by each location</td>
    </tr>
</table>

<h2><u>Monte Carlo For Estimation of Arrival Rates at the Stations</u></h2>
Apply Monte Carlo for estimating arrival rates at each station locations. 
<br><br>
<table>
    <tr>
        <th>Variables</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>iterations</td>
        <td>Number of polls generated</td>
    </tr>
    <tr>
        <td>stations_lat_longs</td>
        <td>List containing lat-longs of stations in the form [(lat1, long1),(lat2, long2),.....,(latN, longN)]</td>
    </tr>
    <tr>
        <td>center</td>
        <td>lat-longs of the centre of the circle in which random origin and destination lat-longs are generated</td>
    </tr>
    <tr>
        <td>stations</td>
        <td>Dict containing the number of vehicles received by each location</td>
    </tr>
</table>

In [2]:
# IMPORTS REQUIRED

import numpy as np
from random import choice
from tqdm.notebook import trange
import time
import matplotlib.pyplot as plt
import multiprocessing as mp
import pandas as pd
from datetime import datetime

# for GA
from geneticalgorithm import geneticalgorithm as ga

# for PSO
from pyswarms.single.global_best import GlobalBestPSO
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)

# for PuLP
import pulp

from collections import Counter
from itertools import combinations

# for finding factorial of a numpy array 
from scipy.special import factorial

%matplotlib inline

In [3]:
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2) :
    """
    Using numpy vectorization made the poll around 10x faster 🥳
    For estimating distances between 2 lat-longs based on Haversine formula
    lat1, lon1 are numpy arrays
    lat2, lon2 are values
    """
    r = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)

    delta_phi, delta_lambda = np.radians(lat2 - lat1), np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2

    return (r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))).tolist()   # in km   

In [4]:
def randomGeo(center, radius) :
    """
    Generating random lat-long in a circular region around a given center and radius
    """
    y0, x0 = center["latitude"], center["longitude"]

    rd = radius / 111300

    u, v = np.random.rand(), np.random.rand()

    w = rd * np.sqrt(u);
    t = 2 * np.pi * v;
    x = w * np.cos(t);
    y = w * np.sin(t);

    return { 'latitude': y + y0, 'longitude': x + x0 }

In [5]:
def poll(junk) :
    # junk argument is there for the sake of running this function on multi-core
    """
    Criteria for station selection :
    1. Min. distance between vehicle and station
    2. Min. distance between station and workplace
    3. Min. sum of distances between vehicle - station - workplace

    Criteria for station-selection-criteria selection
    1. Battery percentage remaining

    Criteria for charging type selection :
    1. Battery percentage remaining
    2. Availability of time
    """
    radius = 20000   # 20km radius
    myLocation = randomGeo(center, radius)  # random origin
    workPlaceLocation = randomGeo(center, radius)  # random destination

    battery = choice(list(range(20, 71)))  # randomly choosing a battery
    charging_choice = np.random.randint(40, high=51)   # if charging is less than this, the guy will go for fast charging, else slow charging

    # criteria selection for station selection by driver based on battery percentage
    if battery in range(20, 41) :
        # if the battery is considerably low, then driver would like to charge the vehicle as soon as possible
        station_selection_criteria = "min_b/w_you_and_station"
    elif battery in range(60, 71) :
        # if the battery is high, the driver would like to spend the remaining charge first, so that he doesnot have to charge the vehicle again at the destination
        station_selection_criteria = "min_b/w_station_and_work"
    else :
        # if the battery is satisfactory, the driver chooses the shortest path possible to the workplace
        station_selection_criteria = "min_sum_of_both"

    # randomly select a criteria

    has_time = choice(["yes", "no"])   # if the driver has time to spare or not

    # making the choice of slow or fast charging based on current battery percentage and availability of time
    # if the driver has time, then he will logically go for slow charger. Why pay extra.
    # if the charge is more than desired threshold, go for slow charger
    pod = "slow" if battery > charging_choice or has_time == "yes" else "fast"
    
    #converting lat and longs into separate numpy arrays
    lat1, lon1 = np.array([i for i, _ in stations_lat_longs]), np.array([j for _, j in stations_lat_longs]) 
    
    # big brain time
    # no need to calculate both the distances everytime, find them according to the condition
    # reduced the time by 40% 🥳
    if station_selection_criteria == "min_b/w_you_and_station" :
        dist1 = getDistanceFromLatLonInKm(lat1, lon1, np.array([myLocation["latitude"]]), np.array([myLocation["longitude"]]))
        return (dist1.index(min(dist1)), pod)

    elif station_selection_criteria == "min_b/w_station_and_work" :
        dist2 = getDistanceFromLatLonInKm(lat1, lon1, np.array([workPlaceLocation["latitude"]]), np.array([workPlaceLocation["longitude"]]))
        return (dist2.index(min(dist2)), pod)

    else :
        dist2 = getDistanceFromLatLonInKm(lat1, lon1, np.array([workPlaceLocation["latitude"]]), np.array([workPlaceLocation["longitude"]]))
        dist1 = getDistanceFromLatLonInKm(lat1, lon1, np.array([myLocation["latitude"]]), np.array([myLocation["longitude"]]))
        summation = [dist1[i] + dist2[i] for i in range(len(dist1))]
        return (summation.index(min(summation)), pod)

In [6]:
iterations = 10**6

# loading station locations and saving them
df = pd.read_csv("../input/location-data-for-charging-station-study/station_locations.csv")
stations_lat_longs = [(i, j) for i, j in zip(df.Lat, df.Long)]

center = {"latitude": 28.61926, "longitude": 77.15464}
stations = {i : {"fast": 0, "slow": 0} for i in range(len(stations_lat_longs))}

t = time.time()
# running the poll on all 4 cores, making a list, converting the list to stations' dictionary
for i, j in mp.Pool().map(poll, list(range(iterations))) :
    stations[i][j] += 1
    
print("Poll completed....\nTime taken:", time.time()-t, "sec")

Poll completed....
Time taken: 35.904236793518066 sec


<hr style="height:2px;">
<br>
<h2><u>Optimization</u></h2>
Here the optization of the locations using Genetic Algorithm begins

<h3><u>CONSTANTS</u></h3>
<style>
    table, th, td {
        border:2px solid green; 
        border-collapse: collapse;
    }
</style>
<table>
    <tr>
        <th>Constants</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>n</td>
        <td>Total number of possible locations being considered</td>
    </tr>
    <tr>
        <td>m</td>
        <td>Total number of optimized locations required</td>
    </tr>
    <tr>
        <td>F_min/S_min</td>
        <td>number of fast/slow charging stations allowed at a location</td>
    </tr>
    <tr>
        <td>f_min/s_min</td>
        <td>Min number of fast/slow charging pods allowed at a charging station</td>
    </tr>
    <tr>
        <td>F_max/S_max</td>
        <td>Max number of fast/slow charging stations allowed at a location</td>
    </tr>
    <tr>
        <td>f_max/s_max</td>
        <td>Max number of fast/slow charging pods allowed at a charging station</td>
    </tr>
    <tr>
        <td>Cfast/Cslown</td>
        <td>Cost of installation of fast/slow charging stations</td>
    </tr>
    <tr>
        <td>CPfast/CPslow</td>
        <td>Fast/Slow charging station power consumption</td>
    </tr>
    <tr>
        <td>Pelec</td>
        <td>per unit cost of electricity</td>
    </tr>
    <tr>
        <td>pf/ps</td>
        <td>utilization rate of fast/slow charging stations</td>
    </tr>
    <tr>
        <td>lambda_f/lambda_s</td>
        <td>Arrival rate of EVs at fast/slow charging stations</td>
    </tr>
    <tr>
        <td>Pof/Pos</td>
        <td>Probability of no EVs waiting in the fast/slow charging stations </td>
    </tr>
</table>

<hr>

<h3><u>VARIABLES</u></h3>

<table>
    <tr>
        <th>Variables</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>p</td>
        <td>index array containing the index of the location selected for optimization, of size m x 1, in the range [0, n-1] </td>
    </tr>
    <tr>
        <td>F/S</td>
        <td>array containing number of fast/slow charging stations at each selected location, of size m x 1, value greater than 0 </td>
    </tr>
    <tr>
        <td>f/s</td>
        <td>array containing number of fast/slow charging pods at each fast/slow charging station at each selected location, value greater than 0, of size m x 1</td>
    </tr>
</table>

<br><b>NOTE:</b> All the fast/slow charging stations at a location have the same number of fast/slow charging pods respectively

<hr>

<h3><u>COMPUTED VARIABLES</u></h3>
<table>
    <tr>
        <th>Computed Variables</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>dist_dict</td>
        <td>dictionary consisting of distances between the charging locations. Of the format {station_index: [distances between the station and other stations]}</td>
    </tr>
    <tr>
        <td>D</td>
        <td>distance matrix, of size m x n-1. of the format [[distances between location 1 and other stations], [distances between location 2 and other locations], ...] </td>
    </tr>
    <tr>
        <td>DD</td>
        <td>reduced distance matrix, of the size m x 1. of the format [[min(distances between location 1 and other stations)], [min(distances between location 1 and other stations)], ......]. Contains minimum distance between locations and the demand points (other station locations) </td>
    </tr>
</table>

<br><b>NOTE: </b>If a station is selected as a preferred locations, all the other charging stations become demand points for it. <br>
<b>REASON: </b>We are considering this for a highway, so the maximum distance that a vehicle has to travel to before getting recharged must be lower than the distances between the stations. <i>2 stations ke beech mein hi toh charging ki jaroorat padegi vehicle ko.</i>


<h3><u><center>Genetic Algorithm Optimization</center></u></h3>

In [7]:
# COST FACTOR
def cost(F, S, f, s) :
    Ff, Ss =  np.dot(F, f.T), np.dot(S, s.T)   # finding the cross product first

    # calculating the respective costs
    cost_installation = Ff * Cfast + Ss * Cslow
    cost_operation = ( Ff * CPfast + Ff * CPslow ) * Pelec

    #total cost
    total_cost = cost_installation + cost_operation

    return total_cost

In [8]:
# ACCESSIBILITY INDEX
def accessibility_index(p) :
    # forming the D matrix
    D = np.array([dist_dict[int(i)] for i in p])
    # forming DD matrix and adding the elements to get 'd'
    d = np.sum(np.min(D, axis=1).reshape((m,1)))  

    # accessibility index
    A = 1/abs(d)

    return -A

In [9]:
# fetching arrival rates from the stations dict generated previously
# in vehicles/hr
arrival_rate_fast, arrival_rate_slow = [stations[i]["fast"]//2500 for i in stations.keys()], [stations[i]["slow"]//2500 for i in stations.keys()]

# Here we have reduced the arrival_rates by a factor of 5000 as the sample size is very large and does not depict a real life scenario

# hyper-parameter. Will tweak this later
# this is the average time taken by 
charging_rate_fast, charging_rate_slow = 20, 150  # ( minutes for 20% to full charge)  # more options 240min, 300min

mu_fast, mu_slow = (1/charging_rate_fast), (1/charging_rate_slow)

<center><h2><b><u>M/M/N Queueing theory</u></b></h2>

Link : <a href="http://courses.washington.edu/cee587/Readings/Chapter%205%20of%20the%20Mannering%20Book.pdf">Click here</a>

<h4><u>Variables : </u></h4>
<table>
    <tr>
        <th>Variable</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>arrival_rate</td>
        <td>list containing arrival rates for all the stations</td>
    </tr>
    <tr>
        <td>P<sub>o</sub></td>
        <td>Probability of having no vehicles in the station</td>
    </tr>
    <tr>
        <td>P<sub>n</sub></td>
        <td>Probability of n vehicles in the station</td>
    </tr>
    <tr>
        <td>Lambda</td>
        <td>Arrival rate (veh/min)</td>
    </tr>
    <tr>
        <td>mu</td>
        <td>Departure rate (veh/min). Can be calculates as 60/charging rate</td>
    </tr>
    <tr>
        <td>rho</td>
        <td>Traffic Intensity (lambda/mu)</td>
    </tr>
    <tr>
        <td>Q_bar</td>
        <td>average queue length (veh)</td>
    </tr>
</table>
</center>

In [10]:
# Without numpy vectorization. Slower than V.2
def waiting_time_computation(p, F, S, f, s) :
    """
    Waiting time estimation function version 1. Uses dumb list comprehension and explicit for loops. Slower😔
    """
    pods_fast, pods_slow = [int(i*j) for i,j in zip(F,f)], [int(i*j) for i,j in zip(S,s)]    # total number of pods at each selected location
    lambda_fast = [arrival_rate_fast[int(i)]/60 for i in p]                   
    lambda_slow = [arrival_rate_slow[int(i)]/60 for i in p]

    total_waiting_time = 0   

    for i in range(m) :
        try :
            rho_fast, rho_slow = lambda_fast[i]/mu_fast, lambda_slow[i]/mu_slow

            M_fast, M_slow = pods_fast[i], pods_slow[i]

            one_minus_rho_by_N_fast, one_minus_rho_by_N_slow = 1 - (rho_fast/M_fast), 1 - (rho_slow/M_slow)     # 1 - rho/N

            #  this factor must be positive
            penalty = 10**30 if one_minus_rho_by_N_fast < 0 or one_minus_rho_by_N_slow < 0 else 0

            denominator_fast = sum([rho_fast**j / np.math.factorial(j) for j in range(M_fast)]) + ( rho_fast**M_fast / (np.math.factorial(M_fast) * one_minus_rho_by_N_fast))

            denominator_slow = sum([rho_slow**j / np.math.factorial(j) for j in range(M_slow)]) + ( rho_slow**M_slow / (np.math.factorial(M_slow) * one_minus_rho_by_N_slow))

            Po_fast, Po_slow = 1 / denominator_fast, 1 / denominator_slow 
    
            Q_bar_fast = Po_fast * rho_fast**(M_fast+1) / (np.math.factorial(M_fast) * M_fast * (one_minus_rho_by_N_fast ** 2))   # average queue length at fast pods

            Q_bar_slow = Po_slow * rho_slow**(M_slow+1) / (np.math.factorial(M_slow) * M_slow * (one_minus_rho_by_N_slow ** 2))

            # waiting time calculations
            t_fast = (rho_fast + Q_bar_fast) / lambda_fast[i]
            t_slow = (rho_slow + Q_bar_slow) / lambda_slow[i]

            total_waiting_time += t_fast + t_slow + penalty

        except :
            total_waiting_time += 10**30

    return total_waiting_time

In [11]:
def find_denominator_part_1(rho, pods) :
    """
    For finding the first part of the sum of the denominator value used in the waiting time estimation
    """
    # maximum value for preventing the array from becoming jaggered because of the range difference
    maxi = np.max(pods)

    # ranged array creation, uses leading zeros as mask to remove jaggeredness
    arr = np.array([list(range(i)) + [0]*(maxi-i) for i in pods], dtype="int")

    # to get rid of the leading zeros in the array each adding 1 in the sum
    diff = np.array([maxi]) - pods
    cols = rho.shape[0]
    
    return np.sum((rho.reshape((cols,1)) ** arr)/factorial(arr), axis=1) - diff
     
    
# uses numpy vectorization. around 15% faster than V.1. 
def waiting_time_computation2(p, F, S, f, s) :
    """
    Waiting time estimation function version 2. Uses numpy vectorization. Faster 😃
    """
    pods_fast, pods_slow = (F * f).astype("int"), (S * s).astype("int")    # total number of pods at each selected location
    lambda_fast = np.array([arrival_rate_fast[int(i)]/60 for i in p])               
    lambda_slow = np.array([arrival_rate_slow[int(i)]/60 for i in p])

    total_waiting_time = 0   
    rho_fast, rho_slow = lambda_fast/mu_fast, lambda_slow/mu_slow
    
    one_minus_rho_by_N_fast, one_minus_rho_by_N_slow = 1 - (rho_fast/pods_fast), 1 - (rho_slow/pods_slow)     # 1 - rho/N
    
    #  this factor must be positive
    if (one_minus_rho_by_N_fast < 0).any() or (one_minus_rho_by_N_slow < 0).any() :
        return 10**30
    
    factorial_fast, factorial_slow = factorial(pods_fast), factorial(pods_slow)
    
    denominator_fast = find_denominator_part_1(rho_fast, pods_fast) + ( (rho_fast**pods_fast) / (factorial_fast * one_minus_rho_by_N_fast))
    denominator_slow = find_denominator_part_1(rho_slow, pods_slow) + ( (rho_slow**pods_slow) / (factorial_slow * one_minus_rho_by_N_slow))
   
    Po_fast, Po_slow = 1 / denominator_fast, 1 / denominator_slow 
    
    Q_bar_fast = Po_fast * rho_fast**(pods_fast+1) / (factorial_fast * pods_fast * (one_minus_rho_by_N_fast ** 2))   # average queue length at fast pods
    Q_bar_slow = Po_slow * rho_slow**(pods_slow+1) / (factorial_slow * pods_slow * (one_minus_rho_by_N_slow ** 2))

    # waiting time calculations
    t_fast = np.sum((rho_fast + Q_bar_fast) / lambda_fast)
    t_slow = np.sum((rho_slow + Q_bar_slow) / lambda_slow)
    
    return t_fast + t_slow

In [12]:
def optimization_GA(X) :
    # format of X
    # X = [p, F, S, f, s]
    # total variables to optimize = 5m
    p, F, S, f, s = X[:m], X[m:2*m], X[2*m:3*m], X[3*m:4*m], X[4*m:]

    penalty = 0
    # check if any 2 same stations are selected 
    if Counter(p).most_common(1)[0][1] > 1 :
        penalty = 10**30    # keeping the penalty absurdly high 

    # calling all the functions and calculating model cost
    # reducing cost by a factor of 10**6 to normalize it
    # accessibility index increased by a factor of 10
    # waiting time reduced by a factor of 100
    total_cost_to_minimize = abs(cost(F, S, f, s)/(10**6)) + abs(accessibility_index(p)*10) + abs(waiting_time_computation2(p, F, S, f, s)/100) + penalty

    return total_cost_to_minimize

<center><h3><u>BOUNDARIES</u></h3></center>
<table>
    <tr>
        <td>p</td>
        <td>[0, n-1] for each location </td>
    </tr>
    <tr>
        <td>F</td>
        <td>[F_min, F_max] for each location</td>
    </tr>
    <tr>
        <td>S</td>
        <td>[S_min, S_max] for each location</td>
    </tr>
    <tr>
        <td>f</td>
        <td>[f_min, f_max] for each location</td>
    </tr>
    <tr>
        <td>s</td>
        <td>[s_min, s_max] for each location</td>
    </tr>
</table>

In [13]:
# CONSTANTS
n = 14   # this will change as per the locations
m = 5
Cfast, Cslow = 3000, 2500   # in $
CPfast, CPslow = 50, 19.2  # in kW
Pelec = 65   # in $/MWh
F_min, S_min, f_min, s_min = 2, 2, 4, 4
F_max, S_max, f_max, s_max  = 6, 10, 6, 10

# boundaries for all the variables
varBound = np.array([[0, n-1]]*m + [[F_min, F_max]]*m + [[S_min, S_max]]*m + [[f_min, f_max]]*m + [[s_min, s_max]]*m, dtype="int")

# distances
dist= [5.3, 9.5, 3.7, 10.1, 6.6, 7, 1.2, 5.5, 2.8, 1.7, 3.1, 5.1, 1.1]

dist_dict = {i: [] for i in range(n)}

dist_dict[0] += list(np.cumsum(dist))   # for 0th station

for i in range(1, n) :
    dist_dict[i] += list(np.cumsum(dist[i-1::-1]))[::-1] + list(np.cumsum(dist[i:]))

<p><b><u>PLAN</u></b></p>
<ul>
    <li>Will run the model 100 times, capture all the results, find the most common results, the one with the lowest cost wins.
    <li>Then compare different station counts for those selected locations
    <li>Is time consuming. Will take around 5hr to complete

In [14]:
points, F, S, f, s, objective = [], [], [], [], [], []

for _ in range(100) :
    # Defining the optimization model 
    model = ga(function=optimization_GA, variable_type="int", variable_boundaries=varBound, dimension=5*m, 
               convergence_curve=False)
    # run the model
    model.run()
    
    dic = model.output_dict
    dic["variable"] = list(map(str, list(map(int, dic["variable"]))))   # convert all the variables to string so as to join them later
    
    points.append(", ".join(dic["variable"][:m]))
    F.append(", ".join(dic["variable"][m:2*m]))
    S.append(", ".join(dic["variable"][2*m:3*m]))
    f.append(", ".join(dic["variable"][3*m:4*m]))
    s.append(", ".join(dic["variable"][4*m:]))
    objective.append(dic["function"])
    

pd.DataFrame({"points" : points, "F": F, "S": S, "f": f, "s": s, "objective" : objective}).to_csv("Results.csv", index=False)

__________________________________________________ 0.2% GA is running...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


 The best solution found:
 [8. 4. 5. 2. 7. 2. 2. 2. 2. 2. 5. 8. 8. 6. 4. 4. 4. 4. 4. 4. 9. 8. 8. 9.
 6.]

 Objective function:
 9.937301110971728
 The best solution found:
 [4. 2. 5. 8. 7. 2. 2. 2. 2. 2. 8. 6. 8. 9. 6. 4. 4. 4. 4. 4. 8. 9. 8. 5.
 4.]

 Objective function:
 9.937301110971728
 The best solution found:
 [8. 2. 4. 7. 5. 2. 2. 2. 2. 2. 5. 9. 9. 6. 8. 4. 4. 4. 4. 4. 9. 6. 7. 4.
 8.]

 Objective function:
 9.939490420287449
 The best solution found:
 [8. 5. 7. 2. 4. 2. 2. 2. 2. 2. 9. 8. 6. 9. 8. 4. 4. 4. 4. 4. 5. 8. 4. 6.
 8.]

 Objective function:
 9.937301110971728
 The best solution found:
 [7. 8. 4. 5. 2. 2. 2. 2. 2. 2. 3. 9. 8. 8. 6. 4. 4. 4. 4. 4. 8. 5. 8. 8.
 9.]

 Objective function:
 9.937301110971731
 The best solution found:
 [2. 7. 8. 5. 4. 2. 2. 2. 2. 2. 9. 4. 5. 8. 8. 4. 4. 4. 4. 4. 6. 6. 9. 8.
 8.]

 Objective function:
 9.937301110971728
 The best solution found:
 [4. 2. 7. 8. 5. 2. 2. 2. 2. 2. 8. 9. 4. 5. 7. 4. 4. 4. 4. 4. 8. 6. 6. 9.
 9.]

 Objective functio

In [15]:
df = pd.read_csv("./Results.csv")
df

,points,F,S,f,s,objective
0,"8, 4, 5, 2, 7","2, 2, 2, 2, 2","5, 8, 8, 6, 4","4, 4, 4, 4, 4","9, 8, 8, 9, 6",9.937301
1,"4, 2, 5, 8, 7","2, 2, 2, 2, 2","8, 6, 8, 9, 6","4, 4, 4, 4, 4","8, 9, 8, 5, 4",9.937301
2,"8, 2, 4, 7, 5","2, 2, 2, 2, 2","5, 9, 9, 6, 8","4, 4, 4, 4, 4","9, 6, 7, 4, 8",9.939490
3,"8, 5, 7, 2, 4","2, 2, 2, 2, 2","9, 8, 6, 9, 8","4, 4, 4, 4, 4","5, 8, 4, 6, 8",9.937301
4,"7, 8, 4, 5, 2","2, 2, 2, 2, 2","3, 9, 8, 8, 6","4, 4, 4, 4, 4","8, 5, 8, 8, 9",9.937301
...,...,...,...,...,...,...
95,"8, 2, 7, 5, 4","2, 2, 2, 2, 2","5, 6, 3, 8, 8","4, 4, 4, 4, 4","9, 9, 8, 8, 8",9.937301
96,"4, 7, 8, 5, 2","2, 2, 2, 2, 2","8, 4, 5, 8, 6","4, 4, 4, 4, 4","8, 6, 9, 8, 9",9.937301
97,"7, 8, 2, 5, 4","2, 2, 2, 2, 2","4, 9, 6, 8, 8","4, 4, 4, 4, 4","6, 5, 9, 8, 8",9.937301
98,"4, 8, 7, 2, 5","2, 2, 2, 2, 2","8, 9, 3, 6, 8","4, 4, 4, 4, 4","8, 5, 8, 9, 8",9.937301


<center>
    <h3><u>Particle Swarm Optimization</u></h3>
    Have to heavily use numpy vectorization to make it fast.
</center>

In [16]:
# COST FACTOR
def cost_PSO(F, S, f, s) :
    Ff, Ss =  np.sum(F * f, axis=1), np.sum(S * s, axis=1)   

    # calculating the respective costs
    cost_installation = Ff * Cfast + Ss * Cslow
    cost_operation = ( Ff * CPfast + Ff * CPslow ) * Pelec

    #total cost
    total_cost = cost_installation + cost_operation

    return total_cost

In [17]:
# ACCESSIBILITY INDEX
def accessibility_index_PSO(p) :
    # forming a 3D matrix for Distances 
    D = np.array([ [dist_dict[int(j)] for j in p[i]] for i in range(n_particles) ])
    
    # forming DD matrix and adding the elements to get 'd'
    d = np.sum(np.min(D, axis=2), axis=1)
    
    A = 1 / np.abs(d)  # accessibilty index array for all the particles

    return -A

In [18]:
def denominator_part_1_3D(rho ,pods) :
    """
    returns first part of the sum of the denominator used in waiting time estimation. 
    Uses numpy vectorization. 2x faster than normal list comprehension 🥳
    """
    maxi = np.max(pods)

    arr_3d = np.array([[list(range(i)) + [0]*(maxi-i) for i in x] for x in pods], dtype="int")
    
    return np.sum(np.repeat(rho[:,:,np.newaxis], 9, axis=2) ** arr_3d / factorial(arr_3d), axis=2) - (np.array(maxi)-pods)


def waiting_time_computation_PSO(p, F, S, f, s) :
    
    pods_fast, pods_slow = np.array(F*f, int), np.array(S*s, int)  # total number of pods at each selected location
    
    # this is 2x faster than initializing them separately
    lambda_fast = lambda_slow = np.zeros((n_particles, m))
    
    for idx, val in enumerate(p) :
        lambda_fast[idx] = np.array([arrival_rate_fast[int(i)]/60 for i in val])                
        lambda_slow[idx] = np.array([arrival_rate_slow[int(i)]/60 for i in val])
    
    rho_fast, rho_slow = lambda_fast/mu_fast, lambda_slow/mu_slow

    one_minus_rho_by_N_fast, one_minus_rho_by_N_slow = 1 - (rho_fast/pods_fast), 1 - (rho_slow/pods_slow)     # 1 - rho/N

    # return a heavy penalty if the above factor is negative
    if (one_minus_rho_by_N_fast < 0).any() or (one_minus_rho_by_N_slow < 0).any() :
        return np.array([10**40]*n_particles) 

    # this factorial is going to be used in the later function
    fast_factorial, slow_factorial = factorial(pods_fast), factorial(pods_slow)
    
    """
    # Spent 2 hrs on this fucker but to no avail. had to go for the dumb way 😡
    denominator_fast = np.array([ [ np.sum(np.array(rho_fast[i][j])**np.arange(pods_fast[i][j]) / factorial(np.arange(pods_fast[i][j]))) for j in range(m)] for i in range(n_particles) ])  + ( rho_fast**pods_fast / (fast_factorial * one_minus_rho_by_N_fast))
    denominator_slow = np.array([ [ np.sum(np.array(rho_slow[i][j])**np.arange(pods_slow[i][j]) / factorial(np.arange(pods_slow[i][j]))) for j in range(m)] for i in range(n_particles) ])  + ( rho_slow**pods_slow / (slow_factorial * one_minus_rho_by_N_slow))
    """
    # Finally found a way to somewhat vectorize the problem. Made it 2x faster 😃
    denominator_fast = denominator_part_1_3D(rho_fast ,pods_fast) + ( rho_fast**pods_fast / (fast_factorial * one_minus_rho_by_N_fast))
    denominator_slow = denominator_part_1_3D(rho_slow ,pods_slow)  + ( rho_slow**pods_slow / (slow_factorial * one_minus_rho_by_N_slow))
    
    Q_bar_fast = Po_fast * rho_fast**(pods_fast+1) / (fast_factorial * pods_fast * (one_minus_rho_by_N_fast ** 2))   # average queue length at fast pods

    Q_bar_slow = Po_slow * rho_slow**(pods_slow+1) / (slow_factorial * pods_slow * (one_minus_rho_by_N_slow ** 2))

    # waiting time calculations
    t_fast = (rho_fast + Q_bar_fast) / lambda_fast
    t_slow = (rho_slow + Q_bar_slow) / lambda_slow

    waiting_time = t_fast + t_slow + penalty

    return np.abs(np.sum(waiting_time, axis=1))

In [19]:
def optimization_PSO(X) :
    # X is of shape (n_particles x dimensions)
    # format of X
    # X = [[p, F, S, f, s]]* n_particles
    # total variables to optimize = 5m
    
    
    # checking if all the values present are natural numbers or not. If decimal exists, return an extremely large penalty
    """
    if ( (X*10)%10 != 0 ).any() :
        return np.array([10**40]*n_particles) 
    """
    X = np.round(X).astype("int")
    
    p, F, S, f, s = X[:, :m], X[:, m:2*m], X[:, 2*m:3*m], X[:, 3*m:4*m], X[:, 4*m:]
    
    
    # checking if any 2 same stations are selected for each and every particle
    if ( p - p[:,0].reshape((p.shape[0],1)) == 0 ).any() :
        return np.array([10**40]*n_particles)     # returning the penalty and halting the process here only
    
    # calling all the functions and calculating model cost
    # reducing cost by a factor of 10**6 to normalize it
    # accessibility index increased by a factor of 10
    # waiting time reduced by a factor of 100
    total_cost_to_minimize = cost_PSO(F, S, f, s)/(10**6) + accessibility_index_PSO(p)*10 + waiting_time_computation_PSO(p, F, S, f, s)/100 

    return total_cost_to_minimize

# boundaries for all the variables
lb = np.array([0]*m + [F_min]*m + [S_min]*m + [f_min]*m + [s_min]*m)
ub = np.array([n-1]*m + [F_max]*m + [S_max]*m + [f_max]*m + [s_max]*m)

# hyperparameters
options = {"c1":1, "c2":0.7, "w":0.9}

# CONSTANTS
n_particles = 50

optimizer = GlobalBestPSO(n_particles = n_particles, dimensions=5*m, options=options, bounds=(lb, ub))

# uses an array of size (n_particles x dimensions)
cost, pos = optimizer.optimize(optimization_PSO, 1000)